In [1]:
from adam_impact_study.analysis.collect import collect_all_results
from adam_impact_study.analysis.main import run_all_analysis, summarize_impact_study_results
from adam_impact_study.analysis.plots import make_analysis_plots, plot_ip_over_time
import pathlib

In [2]:
run_dir = "march_run/"
out_dir = "march_analysis/"

In [3]:
run_all_analysis(run_dir, out_dir, summary_plots=True, individual_plots=True)

Orbit I00167_b002_v000000_2065-2075 has unlinked observations
Orbit I00090_b009_v000000_2035-2045 has incomplete windows
Orbit I00000_b005_v000000_2065-2075 has no observations
Orbit I00272_b011_v000000_2095-2105 has incomplete windows
Orbit I00002_b002_v000000_2065-2075 has no observations


INFO:adam_impact_study.analysis.main:Saved impact study results to /Users/aleck/Code/adam_impacts_study/march_analysis
INFO:adam_impact_study.analysis.plots:Generated warning time histogram
INFO:adam_impact_study.analysis.plots:Generated realization time histogram
INFO:adam_impact_study.analysis.plots:Generated discoveries by diameter plot
INFO:adam_impact_study.analysis.plots:Generated runtime by diameter plot
INFO:adam_impact_study.analysis.plots:Generated incomplete by diameter plot
INFO:adam_impact_study.analysis.plots:Orbit ID Plotting: I00099_b009_v000000_2105-2115
INFO:adam_impact_study.analysis.plots:Orbit ID Plotting: I00149_b007_v000000_2105-2115
INFO:adam_impact_study.analysis.plots:Orbit ID Plotting: I00169_b011_v000000_2085-2095
INFO:adam_impact_study.analysis.plots:Orbit ID Plotting: I00119_b007_v000000_2095-2105
INFO:adam_impact_study.analysis.plots:Orbit ID Plotting: I00090_b009_v000000_2035-2045
INFO:adam_impact_study.analysis.plots:Orbit ID Plotting: I00189_b009_v0000

In [4]:
impactor_orbits, observations, timings, window_results = collect_all_results("march_run/")

In [5]:
print(impactor_orbits)
print(observations)
print(timings)
print(window_results)
window_results.orbit_id.unique()


ImpactorOrbits(size=14)
Observations(size=1231)
ResultsTiming(size=14)
WindowResult(size=573)


[
  "I00099_b009_v000000_2105-2115",
  "I00149_b007_v000000_2105-2115",
  "I00169_b011_v000000_2085-2095",
  "I00119_b007_v000000_2095-2105",
  "I00090_b009_v000000_2035-2045",
  "I00189_b009_v000000_2085-2095",
  "I00000_b005_v000000_2035-2045",
  "I00272_b011_v000000_2095-2105",
  "I00001_b008_v000000_2115-2125",
  "I00000_b011_v000000_2045-2055",
  "I00201_b008_v000000_2025-2035"
]

In [6]:
summary_results = summarize_impact_study_results(
    impactor_orbits, observations, timings, window_results, out_dir,
)

TypeError: summarize_impact_study_results() got an unexpected keyword argument 'summary_plots'

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

summary_results.to_dataframe()

In [ ]:
make_analysis_plots(summary_results, out_dir)

In [ ]:
plot_ip_over_time(impactor_orbits, window_results, pathlib.Path(out_dir).absolute(), survey_start=None)